In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import pathlib

In [2]:
DATA_PATH = "../datasets/Seed_Data.csv"   # update if necessary
KERAS_MODEL_PATH = "seeds_model.keras"
TFLITE_MODEL_PATH = "seeds_model_quantized.tflite"

In [3]:
df = pd.read_csv(DATA_PATH)

In [4]:
print("Loaded:", df.shape)
df = df.dropna()  # safety

Loaded: (210, 8)


In [5]:
X_df = df.drop(columns=["target"])
y_ser = df["target"]

In [6]:
X = X_df.values.astype(np.float32)

In [7]:
le_y = LabelEncoder()
y = le_y.fit_transform(y_ser.astype(str)).astype(np.int32)
num_classes = len(le_y.classes_)
print("Classes:", num_classes, list(le_y.classes_))

Classes: 3 ['0', '1', '2']


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
print("Train/Test shapes:", X_train.shape, X_test.shape)

Train/Test shapes: (168, 7) (42, 7)


In [9]:
def build_inverted_pyramid_multi(input_dim, num_classes):
    inp = layers.Input(shape=(input_dim,), dtype=tf.float32, name="input")
    norm = layers.Normalization(dtype="float32")
    x = norm(inp)   # we'll adapt this layer later

    x = layers.Dense(256, activation="relu", kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(128, activation="relu", kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(64, activation="relu", kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(32, activation="relu", kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(16, activation="relu", kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(8, activation="relu", kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(4, activation="relu", kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(2, activation="relu", kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    # multiclass output: softmax
    out = layers.Dense(num_classes, activation="softmax", dtype="float32")(x)

    model = tf.keras.Model(inputs=inp, outputs=out)
    # attach norm for external adapt
    model.normalization_layer = norm
    return model


In [10]:
model = build_inverted_pyramid_multi(X_train.shape[1], num_classes)
# Adapt normalization on training data
model.normalization_layer.adapt(X_train)

In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 7)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization (Normalization)   │ (None, 7)              │            15 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 8)              │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 4)              │            1

 Total params: 48,054 (187.71 KB)

 Trainable params: 47,019 (183.67 KB)

 Non-trainable params: 1,035 (4.05 KB)

In [12]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=30, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=500,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 201ms/step - accuracy: 0.2679 - loss: 1.3602 - val_accuracy: 0.2143 - val_loss: 1.1005
Epoch 2/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2440 - loss: 1.3847 - val_accuracy: 0.3333 - val_loss: 1.0998
Epoch 3/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.3690 - loss: 1.2076 - val_accuracy: 0.3333 - val_loss: 1.1000
Epoch 4/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.3393 - loss: 1.2450 - val_accuracy: 0.3333 - val_loss: 1.1004
Epoch 5/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3988 - loss: 1.1722 - val_accuracy: 0.3333 - val_loss: 1.1009
Epoch 6/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3631 - loss: 1.1730 - val_accuracy: 0.3333 - val_loss: 1.1014
Epoch 7/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3631 - loss: 1.1394 - val_accuracy: 0.3333 - val_loss: 1.1016
Epoch 8/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3988 - loss: 1.0927 - val_accuracy: 0.3333 - val_los

In [13]:
model.save(KERAS_MODEL_PATH)
print("Saved Keras model to:", KERAS_MODEL_PATH)

Saved Keras model to: seeds_model.keras


In [14]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
y_probs = model.predict(X_test, verbose=0)  # shape (N, num_classes)
y_pred = np.argmax(y_probs, axis=1)
print("\nKeras model accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))


Keras model accuracy: 0.9047619047619048
              precision    recall  f1-score   support

           0     1.0000    0.7143    0.8333        14
           1     0.8235    1.0000    0.9032        14
           2     0.9333    1.0000    0.9655        14

    accuracy                         0.9048        42
   macro avg     0.9190    0.9048    0.9007        42
weighted avg     0.9190    0.9048    0.9007        42



In [15]:
def representative_dataset_full():
    # yield full training set, one row at a time
    for i in range(X_train.shape[0]):
        sample = X_train[i:i+1].astype(np.float32)   # shape (1, features)
        yield [sample]

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_full

In [17]:
print("\nConverting to TFLite (this may take a moment)...")
tflite_model = converter.convert()
with open(TFLITE_MODEL_PATH, "wb") as f:
    f.write(tflite_model)
print("Saved TFLite quantized model to:", TFLITE_MODEL_PATH)



Converting to TFLite (this may take a moment)...
INFO:tensorflow:Assets written to: C:\Users\pushp\AppData\Local\Temp\tmpyybfiyyw\assets


INFO:tensorflow:Assets written to: C:\Users\pushp\AppData\Local\Temp\tmpyybfiyyw\assets


Saved artifact at 'C:\Users\pushp\AppData\Local\Temp\tmpyybfiyyw'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 7), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2441118327696: TensorSpec(shape=(1, 7), dtype=tf.float32, name=None)
  2441118329616: TensorSpec(shape=(1, 7), dtype=tf.float32, name=None)
  2441112300112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2441112302800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2441112299728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2441112301456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2441112301264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2441112302416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2441112299344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2441112300688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2441112302224: T

e:\AgroML_Hub\models\.venv\Lib\site-packages\tensorflow\lite\python\convert.py:863: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Saved TFLite quantized model to: seeds_model_quantized.tflite


In [18]:
def evaluate_tflite_multiclass(tflite_path, X_data, y_true):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    in_scale, in_zero_point = input_details[0].get("quantization", (0.0, 0))
    out_scale, out_zero_point = output_details[0].get("quantization", (0.0, 0))

    preds = []
    for i in range(X_data.shape[0]):
        inp = X_data[i:i+1].astype(np.float32)  # shape (1,features)

        # quantize input if required
        if in_scale and in_scale != 0:
            q = (inp / in_scale + in_zero_point).round().astype(input_details[0]["dtype"])
            interpreter.set_tensor(input_details[0]["index"], q)
        else:
            interpreter.set_tensor(input_details[0]["index"], inp.astype(input_details[0]["dtype"]))

        interpreter.invoke()
        out = interpreter.get_tensor(output_details[0]["index"])  # shape (1, num_classes) possibly quantized

        # dequantize output if needed
        if out_scale and out_scale != 0:
            out_deq = (out.astype(np.float32) - out_zero_point) * out_scale
        else:
            out_deq = out.astype(np.float32)

        pred = int(np.argmax(out_deq, axis=1)[0])
        preds.append(pred)

    preds = np.array(preds, dtype=int)
    print("\nTFLite evaluation:")
    print("Accuracy:", accuracy_score(y_true, preds))
    print(classification_report(y_true, preds, digits=4))

In [19]:
evaluate_tflite_multiclass(TFLITE_MODEL_PATH, X_test, y_test)

# 11) Done: print helpful info
print("\nDone. Keras saved to:", KERAS_MODEL_PATH, "TFLite saved to:", TFLITE_MODEL_PATH)
print("Use le_y.inverse_transform([pred]) to map numeric labels back to original class values.")


TFLite evaluation:
Accuracy: 0.8809523809523809
              precision    recall  f1-score   support

           0     1.0000    0.6429    0.7826        14
           1     0.8235    1.0000    0.9032        14
           2     0.8750    1.0000    0.9333        14

    accuracy                         0.8810        42
   macro avg     0.8995    0.8810    0.8731        42
weighted avg     0.8995    0.8810    0.8731        42


Done. Keras saved to: seeds_model.keras TFLite saved to: seeds_model_quantized.tflite
Use le_y.inverse_transform([pred]) to map numeric labels back to original class values.


e:\AgroML_Hub\models\.venv\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
